<a href="https://colab.research.google.com/github/bhavinbajaj/bhavin15/blob/main/code_for_SAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# **PART1:QUESTION TO EMBEDDING**

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install gensim

In [ ]:
import gensim

In [ ]:
model_path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'            # Path where the model is stored
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)            # Loading the model using gensim

In [ ]:
import json
import h5py
import numpy as np
import copy
from random import shuffle, seed
import sys
import os.path
import argparse
import glob
import numpy as np
import scipy.io
import pdb
import string
import h5py
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import gensim
import json
import re
import cv2
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import os
import re
import sys
import json
import numpy as np
import h5py
from nltk.tokenize import word_tokenize

# Define the extract_feat function
def extract_feat(doc):
    '''
    Input: A sentence
    Output: Word Embedding of the sentence


    '''
    feat = []
    for word in doc:
        try:
            feat.append(model_w2v[word])
        except:
            pass
    return feat

# Define the tokenize function
def tokenize(sentence):
    ''' Perform Tokenization '''
    return [i for i in re.split(r"([-.\"',:? !\$#@~()*&\^%;\[\]/\\\+<>\n=])", sentence) if i != '' and i != ' ' and i != '\n']

# Define the prepro_question function
def prepro_question(imgs, method):
    # preprocess all the questions
    print('example processed tokens:')
    '''
    Input: The question from trainset.json
    Performs tokenization and lowering of the question
    Output: Embedded version of the question


    '''
    for i, img in enumerate(imgs):
        s = img['question'].lower()
        if method == 'nltk':
            txt = word_tokenize(str(s).lower())
        else:
            txt = tokenize(s)
        img['processed_tokens'] = txt
        if i < 10:
            print(txt)
        if i % 1000 == 0:
            sys.stdout.write("processing %d/%d (%.2f%% done)   \r" % (i, len(img), i * 100.0 / len(imgs)))
            sys.stdout.flush()
    return imgs

# Define the get_top_answers function
def get_top_answers(imgs, num_ans):
    """
    Print the questions and returns the time, one answer is repeated
    """
    counts = {}
    for img in imgs:
        try:
            ans = img['answer'].lower()  # If the string is a number, it would result into error
        except:
            ans = str(img['answer'])
        counts[ans] = counts.get(ans, 0) + 1

    cw = sorted([(count, w) for w, count in counts.items()], reverse=True)
    print('top answer and their counts:')
    print('\n'.join(map(str, cw[:20])))

    vocab = []
    for i in range(min(num_ans, len(cw))):
        vocab.append(cw[i][1])

    return vocab[:num_ans]

# Define the filter_question function
def filter_question(imgs, atoi):

    new_imgs = []
    for i, img in enumerate(imgs):
        new_imgs.append(img)

    print('question number reduce from %d to %d ' % (len(imgs), len(new_imgs)))
    return new_imgs

# Define manualMap and load data
manualMap = {'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three':
             '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7',
             'eight': '8', 'nine': '9', 'ten': '10'}

imgs_train = json.load(open('/content/drive/MyDrive/trainset.json', 'r'))
num_ans = 1000
top_ans = get_top_answers(imgs_train, num_ans)
atoi = {w: i for i, w in enumerate(top_ans)}  # Word: Count
itoa = {i: w for i, w in enumerate(top_ans)}  # Count: Word
feat_dim = 300  # 300 Dimensional Vector
imgs_data_train = json.load(open('/content/drive/MyDrive/trainset.json', 'r'))
num_ans = 10  # Even 1 should work fine, but I had taken reference from COCO dataset, and hence, 10 (10 represents the top 10 answers to a picture)
method = 'nltk'
max_length = 21  # Max Length of the question
dir_path = "/content/drive/MyDrive/QA"  # The path where we will be storing .h5 file
N = 700

image_path = '/content/drive/MyDrive/VQA_RAD Image Folder.zip'

def save_data():
    for i, img in enumerate(imgs_data_train[:N]):
        img_path = image_path + img['image_name']

        s = img['question']
        print(i, s)  # Print the number and the question
        if method == 'nltk':
            try:
                txt = word_tokenize(str(s).lower())
            except:
                txt = str(s)
        else:
            txt = tokenize(s)

        img['processed_tokens'] = txt
        question_id = img['qid']
        feat = np.array(extract_feat(img['processed_tokens']))
        label_arrays = np.zeros((1, max_length, feat_dim), dtype='float32')
        label_length = min(max_length, len(feat))  # record the length of this sequence
        label_arrays[0, :label_length, :300] = feat
        try:
            ans_arrays = atoi[img['answer'].lower()]
        except:
            ans_arrays = atoi[str(img['answer'])]

        f = h5py.File(os.path.join(dir_path, str(question_id) + '.h5'), "w")
        f.create_dataset("ques_train", dtype='float32', data=label_arrays)
        f.create_dataset("answers", dtype='uint32', data=ans_arrays)
        f.close()
    return

data = save_data()


top answer and their counts:
(884, 'no')
(829, 'yes')
(35, 'right')
(31, 'axial')
(23, 'left')
(15, 'fat')
(14, 'pa')
(12, 'right lung')
(12, 'pancreas')
(12, 'one')
(11, 'left kidney')
(11, 'diffuse')
(10, 'right upper lobe')
(10, 'right sided pleural effusion')
(10, 'right side')
(10, 'ct')
(10, 'brain')
(9, 'lateral ventricles')
(9, 'bilateral')
(8, 'subarachnoid')
0 Are regions of the brain infarcted?
1 Are the lungs normal appearing?
2 Is there evidence of a pneumothorax
3 What type of imaging does this not represent?
4 Is this a MRI of the chest?
5 What is not pictured in this image?
6 Is the trachea midline?
7 Is there evidence of an aortic aneurysm?
8 Where is the abnormality?
9 Is there blunting of the costovertebral angles?
10 Which organ system is abnormal in this image?
11 Where is the pathology in this image?
12 Is there a pneumothorax?
13 What type of imaging is this?
14 What organ system is pictured?
15 Is there a fracture?
16 Is there swelling of the grey matter?
17 Is 

# **PART 2:LOADING AND PREPARING THE DATASETS**

In [ ]:
import json
import h5py
import numpy as np
import copy
from random import shuffle, seed
import sys
import os.path
import argparse
import glob
import numpy as np
import scipy.io
import pdb
import string
import h5py
import nltk
from nltk.tokenize import word_tokenize
import gensim
import json
import re
import cv2
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
import os
import matplotlib.pyplot as plt
dropout_rate = 0.4
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# -------------------------------------------------------------------------------------------------------------
# The below part is same as that of part 1

def extract_feat(doc):
    feat = []
    for word in doc:
        try:
            feat.append(model_w2v[word])
        except:
            pass
    return feat

def tokenize(sentence):
    return [i for i in re.split(r"([-.\"',:? !\$#@~()*&\^%;\[\]/\\\+<>\n=])", sentence) if i!='' and i!=' ' and i!='\n'];

def prepro_question(imgs, method):

    # preprocess all the question
    print('example processed tokens:')
    for i,img in enumerate(imgs):
        s = img['question'].lower()
        if method == 'nltk':
            txt = word_tokenize(str(s).lower())
        else:
            txt = tokenize(s)
        img['processed_tokens'] = txt
        if i < 10: print(txt)
        if i % 1000 == 0:
            sys.stdout.write("processing %d/%d (%.2f%% done)   \r" %  (i, len(img), i*100.0/len(imgs)) )
            sys.stdout.flush()
    return imgs

def get_top_answers(imgs, num_ans):
    counts = {}
    for img in imgs:
        try:
            ans = img['answer'].lower()
        except :
            ans = str(img['answer'])
        counts[ans] = counts.get(ans, 0) + 1 # Frequency count

    cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
    # print('top answer and their counts:')
    # print('\n'.join(map(str,cw[:20])))

    vocab = []
    for i in range(min(num_ans,len(cw))):
        vocab.append(cw[i][1])

    return vocab[:num_ans]


def filter_question(imgs, atoi):
    new_imgs = []
    for i, img in enumerate(imgs):
            new_imgs.append(img)

    print('question number reduce from %d to %d '%(len(imgs), len(new_imgs)))
    return new_imgs
# Old Part finishes
# ---------------------------------------------------------------------------------------------------------------------
# New part starts
# -------------------------------------------------------------------------------------------------------------------
def image_layer(input_shape):
    '''
    Input : Shape of the image
    Output : VGG16 Preprocessing model
    '''
    base_model = tf.keras.applications.VGG16(input_shape=input_shape, include_top=False,weights='imagenet')
    base_model.trainable = False # Do not train it
    x = base_model.layers[-2].output  # Shape would be (28*28*512)
    x = tf.reshape(x , [-1,x.shape[2]*x.shape[1] , x.shape[3]]) # Shape would be (1,784,512)
    x = tf.keras.layers.Dense(1024)
    return x

def vgg_preprocessing(model,image):
  ''' Takes a tensor as an input, and returns a pre processed version of the image'''
  return model(image)
  N=700

def load_data():



        images = []
        questions = []
        answers = []
        ids = []

        #print(start,end)
        #arrs = np.random.randint(0,len(imgs_data_train),batch)
        #data = [imgs_data_train[i] for i in arrs]

        data = imgs_data_train[:N]   # trainset.json
        model = image_layer(input_shape = (448,448,3)) # Making VGG16 Model
        for i,img in enumerate(data):

            img_path = img['image_name']  # Image Name
            question_id = img['qid']      # Question id

            #label_arrays = np.zeros((1, max_length, feat_dim), dtype='float32')

            with h5py.File(os.path.join(dir_path,str(question_id) + '.h5'),'r') as hf:
                question = hf['.']['ques_train'[:N]][()] # Embedded question
                answer = hf['.']['answers'[:N]][()]    # Embedded answer

            image = cv2.imread(os.path.join('/content/drive/MyDrive/VQA_RAD Image Folder',img_path) , cv2.IMREAD_COLOR) # Reading the image
            image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
            image = cv2.resize(image , (448,448)) # Reshape

            '''

            '''
            #image = vgg_preprocessing(model,image)
            # Apply VGG16 Preprocessings

            images.append(image)
            questions.append(np.array(question))
            answers.append(np.array(answer))
            ids.append(question_id)
            if i%100==0:
              print("Processed =>",i,' which is',round(100*i/len(data),2),'%')

        questions = np.reshape(np.array(questions) , [-1,max_length,feat_dim])
        return (np.array(images) , questions ,np.array(answers) , np.array(ids))

imgs_train = json.load(open('/content/drive/MyDrive/trainset.json' , 'r'))
num_ans = 1000
top_ans = get_top_answers(imgs_train, num_ans )
atoi = {w:i for i,w in enumerate(top_ans)}
itoa = {i:w for i,w in enumerate(top_ans)}
feat_dim = 300
imgs_data_train = json.load(open('/content/drive/MyDrive/trainset.json' , 'r'))
num_ans = 10
method = 'nltk'
max_length = 21
dir_path = "/content/drive/MyDrive/QA"
N = 700



In [ ]:
import tensorflow as tf
import numpy as np
import keras,h5py
import pandas as pd
import os
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from keras.applications.vgg16 import VGG16
import keras.activations
import keras.backend as kbe
from keras.callbacks import EarlyStopping
import tensorflow.keras.layers as layers
from keras.layers import Activation, Add, Concatenate, Conv1D, Dense, Dropout, Embedding, Softmax
from keras.layers import Input, GlobalMaxPooling1D, Lambda, Multiply, RepeatVector, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
import pickle
from pprint import pprint



embed_size = 300
q_len = 21
height = 224
width = 224
lstm_units = 256
attention_dim = 512
num_output = 1000
max_questions = 3064

batch_size = 16
lr = 0.001
articles = ['a', 'an', 'the']
manualMap = { 'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three':
                  '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7',
                  'eight': '8', 'nine': '9', 'ten': '10' }


datagen = load_data()   # Load the Data

Processed => 0  which is 0.0 %
Processed => 100  which is 14.29 %
Processed => 200  which is 28.57 %
Processed => 300  which is 42.86 %
Processed => 400  which is 57.14 %
Processed => 500  which is 71.43 %
Processed => 600  which is 85.71 %


### Creating the Dataset: We do this, by renaming the contents of .h5 file

In [ ]:
images,questions,answers,ids =datagen[0],datagen[1],datagen[2],datagen[3]
print("Images have a size of:",images.shape)
print("Questions have a size of:",questions.shape)
print("Answers have a size of:",answers.shape)
print("Ids have a size of:",ids.shape)
dir_path = r'/content/drive/MyDrive/QA' # The directory where the .h5 file for each entry is saved
m = 100
for i in range(images.shape[0]):
    ans_array = answers[i]
    image_array = images[i]
    quest_array = questions[i]
    question_id = ids[i]
    f = h5py.File(os.path.join( dir_path , str(question_id) + '.h5'), "w") # Loading the 'h5 file
    f.create_dataset("ques_train", dtype='float32', data=quest_array) # Question Embedding
    f.create_dataset("image_vector", dtype='float32', data=image_array) # Image Embedding (Not preprocessed)
    f.create_dataset("answers", dtype='uint32', data=ans_array)      # Answers in embedded form
    f.close()
    if i%m ==0:
        print("Processed =>", i,' total percentage =>', round(100*i/images.shape[0],2),' %')
print("Your processing has been done")

Images have a size of: (700, 448, 448, 3)
Questions have a size of: (700, 21, 300)
Answers have a size of: (700,)
Ids have a size of: (700,)
Processed => 0  total percentage => 0.0  %
Processed => 100  total percentage => 14.29  %
Processed => 200  total percentage => 28.57  %
Processed => 300  total percentage => 42.86  %
Processed => 400  total percentage => 57.14  %
Processed => 500  total percentage => 71.43  %
Processed => 600  total percentage => 85.71  %
Your processing has been done


# **PART3:MAKING THE MODEL**

In [ ]:
# link - http://www.cs.virginia.edu/~vicente/vislang/slides/wasimonica.pdf
import os,h5py
# Image Model
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model
import warnings
warnings.filterwarnings('ignore')

'''  Preprocessing with the VGG 16 Model  '''

model = tf.keras.applications.VGG16(include_top=False,weights='imagenet',
        input_shape=(448,448,3))
#print("The Last layer")
last_layer = model.layers[-1].output   # Last layer has an output layer of (14,14,512)
model = Model(model.input,last_layer)
model.trainable = False
# print(model.summary())
def extract_feature(image):
    ''' Preprocessing with VGG Netowrk'''
    image = model(image)
    return image  # Shape is (196,512)

'''  The below model will convert (196,512) to (21,300) (i.e same as the dimension of word embedding)  '''

dimen_red = tf.keras.Sequential()  # Use for converting (196,512) -> (21,300)
dimen_red.add(tf.keras.layers.Conv2D(300,kernel_size=(1,1),input_shape= (14,14,512)))
dimen_red.add(tf.keras.layers.Reshape((196,300)))
dimen_red.add(tf.keras.layers.Permute((2,1)))  # Reshaping about the axis, useful for applying the dense network
dimen_red.add(tf.keras.layers.Dense(21))
dimen_red.add(tf.keras.layers.Permute((2,1)))  # Reshaping about the axis, useful for applying the dense network

train_dir = r'/content/drive/MyDrive/QA/' # Containing .h5 file
images = []
ans = []
ques = []
count = 0
content = os.listdir(train_dir)[:600]   # The GPU Memory became full after this, hence had to take just these much samples :(
length = len(content)
for i in content:
    # Reading the data
    file = h5py.File(train_dir+i)
    images.append(np.array(file['.']['image_vector'][()]))
    ans.append(np.array(file['.']['answers'][()]))
    ques.append(np.array(file['.']['ques_train'][()]))
    count+=1
    if count%100 == 0:
      print("The count is:",count,"and the percentage proportion is:",round(100*count/length,2),'%')
images = tf.convert_to_tensor(np.array(images))   # For the GPU purpose
ans = tf.convert_to_tensor(np.array(ans))
ques = tf.convert_to_tensor(np.array(ques))

The count is: 100 and the percentage proportion is: 16.67 %
The count is: 200 and the percentage proportion is: 33.33 %
The count is: 300 and the percentage proportion is: 50.0 %
The count is: 400 and the percentage proportion is: 66.67 %
The count is: 500 and the percentage proportion is: 83.33 %
The count is: 600 and the percentage proportion is: 100.0 %


In [ ]:
l = []
length = images.shape[0]
for i,j in enumerate(images):
  l.append(model(tf.reshape(j,[1,448,448,3])))  # It was not possible directly on GPU, hence had to use for loop
  if i%100 ==0:
    print("The count is:",i,"and the percentage proportion is:",round(100*i/length,2),'%')
images = tf.convert_to_tensor(np.array(l) )

The count is: 0 and the percentage proportion is: 0.0 %
The count is: 100 and the percentage proportion is: 16.67 %
The count is: 200 and the percentage proportion is: 33.33 %
The count is: 300 and the percentage proportion is: 50.0 %
The count is: 400 and the percentage proportion is: 66.67 %
The count is: 500 and the percentage proportion is: 83.33 %


In [ ]:
l = []
length = images.shape[0]
for i,j in enumerate(images):
  l.append(dimen_red(j))     # Making it to the same shape as that of question embedding
  if i%100 ==0:
    print("The count is:",i,"and the percentage proportion is:",round(100*i/length,2),'%')
images = tf.convert_to_tensor(np.array(l))

The count is: 0 and the percentage proportion is: 0.0 %
The count is: 100 and the percentage proportion is: 16.67 %
The count is: 200 and the percentage proportion is: 33.33 %
The count is: 300 and the percentage proportion is: 50.0 %
The count is: 400 and the percentage proportion is: 66.67 %
The count is: 500 and the percentage proportion is: 83.33 %


In [ ]:
images = tf.reshape(images,[length,21,300])
img = images  #Tensor containing images
que = ques  # Tensor containing question vector
img = img/255.0 # Normalizing

In [ ]:
que.shape,img.shape

(TensorShape([600, 21, 300]), TensorShape([600, 21, 300]))

In [ ]:
''' The below mentioned two Input objects of keras will be useful for making the model '''

ques = tf.keras.layers.Input((21,300))  # Input Model (for ques)
images = tf.keras.layers.Input((21,300)) # Input Model (for images)

In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D,Flatten,Concatenate
from keras.layers import Attention


''' Imagica is for the preprocessing of the image part'''
imagica = Dense(512,activation='tanh')(images)
imagica = Dense(512,activation='tanh')(images)


''' quesa is for the ques layer, which means preprocessing of the question layer'''
quesa = LSTM(512, dropout = 0.3,return_sequences = True,input_shape = (21,300))(ques)
quesa = Dense(512, activation = 'relu')(quesa)
quesa = Dropout(0.3)(quesa)
quesa = Dense(512, activation = 'relu')(quesa)
quesa = Dropout(0.3)(quesa)

concatenated_features = Concatenate()([imagica, quesa])

# Apply attention mechanism
attention = Attention()([concatenated_features, concatenated_features])

# Combine the attention result with the concatenated features
attended_features = Concatenate()([concatenated_features, attention])

# Flatten the attended features
attended_features = Flatten()(attended_features)

# Final output layer
out = Dense(476, activation='softmax')(attended_features)

# ''' Concatenating both image and the question layer'''
# quesa = Concatenate()([quesa,imagica])
# quesa = Flatten()(quesa)
# out = tf.keras.layers.Dense(476,activation='softmax')(quesa) # Final output has 476 different categories, you can check by finding length of uniquue answers :)

In [ ]:
out

<KerasTensor: shape=(None, 476) dtype=float32 (created by layer 'dense_19')>

## Plotting the Model

In [ ]:
pip install pydot graphviz


In [ ]:
model = Model([ques,images],[out])

## Compiling and fitting the model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),loss ='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
answers = tf.keras.utils.to_categorical(ans)
answers.shape

(600, 455)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 21, 300)]            0         []                            
                                                                                                  
 lstm_3 (LSTM)               (None, 21, 512)              1665024   ['input_2[0][0]']             
                                                                                                  
 dense_17 (Dense)            (None, 21, 512)              262656    ['lstm_3[0][0]']              
                                                                                                  
 dropout_6 (Dropout)         (None, 21, 512)              0         ['dense_17[0][0]']            
                                                                                            

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert your data to NumPy arrays
img = np.array(img)
que = np.array(que)
ans = np.array(ans)

# Split your data into training and validation sets
img_train, img_val, que_train, que_val, ans_train, ans_val = train_test_split(img, que, ans, test_size=0.2, random_state=42)

# Assuming you already have your model defined and compiled

# Train the model
model.fit([img_train, que_train], ans_train, epochs=30, batch_size=32, verbose=1, validation_data=([img_val, que_val], ans_val))



Epoch 1/30
15/15 [==============================] - 39s 2s/step - loss: 20.3862 - accuracy: 0.3354 - val_loss: 7.0147 - val_accuracy: 0.4500
Epoch 2/30
15/15 [==============================] - 16s 963ms/step - loss: 3.7116 - accuracy: 0.6104 - val_loss: 6.1944 - val_accuracy: 0.5000
Epoch 3/30
15/15 [==============================] - 12s 759ms/step - loss: 2.2549 - accuracy: 0.7146 - val_loss: 7.3342 - val_accuracy: 0.5250
Epoch 4/30
15/15 [==============================] - 12s 796ms/step - loss: 1.9009 - accuracy: 0.8167 - val_loss: 6.6288 - val_accuracy: 0.5917
Epoch 5/30
15/15 [==============================] - 12s 820ms/step - loss: 1.7293 - accuracy: 0.8208 - val_loss: 8.3138 - val_accuracy: 0.5250
Epoch 6/30
15/15 [==============================] - 13s 900ms/step - loss: 1.5442 - accuracy: 0.8125 - val_loss: 6.9573 - val_accuracy: 0.5833
Epoch 7/30
15/15 [==============================] - 13s 904ms/step - loss: 1.7486 - accuracy: 0.8292 - val_loss: 8.9317 - val_accuracy: 0.5667
E

## Saving the Model and analysing the prediction

In [ ]:
model.save('/content/drive/MyDrive/VQA_Model')

## END OF THE NOTEBOOK :)